In [ ]:
import pandas as pd

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")

In [ ]:
movies[:2]

In [ ]:
credits = pd.read_csv("credits.csv")

In [ ]:
credits[:2]

In [ ]:
movies[:2]

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
credits = credits.rename(columns={'movie_id':'id'})

In [ ]:
df = credits.merge(movies, on='id')

In [ ]:
df[:2]

In [ ]:
vote_count = df['vote_count']

In [ ]:
avg_rating = df['vote_average']

In [ ]:
avg_vote = df['vote_average'].mean()

In [ ]:
avg_vote

In [ ]:
min_votes = df['vote_count'].quantile(0.75)

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(y=df['vote_count'])

In [ ]:
df[df.vote_count==1]

In [ ]:
min_votes

In [ ]:
df['weighted_rating'] = ((avg_rating*vote_count)+(avg_vote*min_votes))/(vote_count+min_votes)

In [ ]:
# Recommend movies based on rating

In [ ]:
df_sorted = df.sort_values('weighted_rating', ascending=False)

In [ ]:
df_sorted[:2]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Recommend movies based on voting and rating (vote count & voting avg)

In [ ]:
plt.figure(figsize=(15,6))
ax1 = sns.barplot(x=df_sorted['weighted_rating'].head(10), y = df_sorted['original_title'].head(10), data = df_sorted)
plt.xlabel('rating')
plt.ylabel('movie_name')
plt.show()

In [ ]:
# Recommend based on popularity

In [ ]:
df_popularity = df.sort_values('popularity', ascending=False)

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(15,6))
ax1 = sns.barplot(x=df_popularity['popularity'].head(10), y = df_popularity['original_title'].head(10))
plt.xlabel('rating')
plt.ylabel('movie_name')
plt.show()

In [ ]:
# Recommend movies based on voting and rating (vote count & voting avg) and popularity --> Weighted_avg

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mx = MinMaxScaler()

In [ ]:
df_normalized = mx.fit_transform(df[['weighted_rating','popularity']])

In [ ]:
df_normalized = pd.DataFrame(df_normalized, columns=[['weighted_rating_norm','popularity_norm']])

In [ ]:
df_normalized[:2]

In [ ]:
df[['weighted_rating_norm','popularity_norm']] = df_normalized

In [ ]:
df[:2]

In [ ]:
df['score'] = df['weighted_rating_norm']*0.5+df['popularity_norm']*0.5

In [ ]:
df['score'][:5]

In [ ]:
df['weighted_rating'][:5]

In [ ]:
df['popularity'][:5]

In [ ]:
score_sorted = df.sort_values('score', ascending=False)

In [ ]:
plt.figure(figsize=(15,6))
ax1 = sns.barplot(x=score_sorted['score'].head(10), y = score_sorted['original_title'].head(10))
plt.xlabel('score')
plt.ylabel('movie_name')
plt.show()

In [ ]:
# Content based recommendation --> Similar items --> Pairwise distance, Jaccard similarity/euclidean/minkowski, cosine similarity, linear and non-linear kernels

In [ ]:
df[:2]

In [ ]:
df['overview'][:5]

In [ ]:
# TF/IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# term frequency --> tf --> How many times a word has occured in a document
# inverse document frequency --> idf --> how many documents has that word

In [ ]:
# get rid of stop words --> this, that , is, was

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
help(TfidfVectorizer)

In [ ]:
# Stemming should be done --> loving, loved, love, lovable --> love

In [ ]:
df1 = df.copy()

In [ ]:
df1['overview'] = df1['overview'].fillna('')

In [ ]:
tfidf_matrix = tfidf.fit_transform(df1['overview'])

In [ ]:
tfidf_matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# we are dealing with huge sparse matrix, and hence let's use a kernel

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

In [ ]:
indices = pd.Series(df1.index, index=df1['title_x']).drop_duplicates()

In [ ]:
indices

In [ ]:
df1.index

In [ ]:
df1.drop('title_y', axis=1, inplace=True)

In [ ]:
cosine_sim[0][:815]

In [ ]:
for ind,val in enumerate("abcd"):
    print(ind,val)

In [ ]:
list(enumerate(cosine_sim[indices['Interstellar']]))                                        

In [ ]:
df1.iloc[4353]

In [ ]:
sorted(list(enumerate(cosine_sim[indices['Interstellar']])), key=lambda x:x[1],reverse=True)                      